In [1]:
import pandas as pd

In [11]:
meta_set = pd.read_pickle('../data/set_with_meta.pickle')
meta_set[:2]

,disease_main,doid,disease,dataset_name,name,samples,platform,chip,normal_samples,norma_,tissue,tissue_main,comments,accession,summary,title,overall_design
0,breast cancer,DOID:0060075,estrogen-receptor positive breast cancer,GSE65194_ER+,Expression profiling of breast cancer samples ...,59,GPL570,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,11,-,breast sample,breast sample,NaN,GSE65194,Transcriptome analysis of 130 breast cancer sa...,Expression profiling of breast cancer samples ...,This dataset contains 178 arrays. 153 arrays w...
1,breast cancer,DOID:0060079,Her2-receptor positive breast cancer,GSE65194_Her2+,Expression profiling of breast cancer samples ...,39,GPL570,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,11,-,breast sample,breast sample,NaN,GSE65194,Transcriptome analysis of 130 breast cancer sa...,Expression profiling of breast cancer samples ...,This dataset contains 178 arrays. 153 arrays w...


In [79]:
meta_set[meta_set['accession'] == 'GSE21422']


,disease_main,doid,disease,dataset_name,name,samples,platform,chip,normal_samples,norma_,tissue,tissue_main,comments,accession,summary,title,overall_design
18,breast cancer,DOID:0060074,ductal carcinoma in situ,GSE21422_DCIS,Expression profiling of human DCIS and invasiv...,9,GPL570,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,5,-,breast sample,breast sample,NaN,GSE21422,"Human healthy tissue samples, DCIS and invasiv...",Expression profiling of human DCIS and invasiv...,"5 healthy tissue samples, 9 DCIS and 5 invasiv..."
19,breast cancer,DOID:3008,invasive ductal carcinoma,GSE21422_IDC,Expression profiling of human DCIS and invasiv...,5,GPL570,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,5,-,breast sample,breast sample,NaN,GSE21422,"Human healthy tissue samples, DCIS and invasiv...",Expression profiling of human DCIS and invasiv...,"5 healthy tissue samples, 9 DCIS and 5 invasiv..."


In [78]:
meta_set['disease'] = meta_set['disease'].map(lambda x: x.strip())

In [7]:
from elasticsearch import Elasticsearch
es = Elasticsearch(host='scraper.now.im', port='8080')

res = es.search(index="series", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total'])

Got 56468 Hits:


In [35]:
def search_phrase(phrase):
#     {
#   "query": {
#     "match": {
#       "title": {
#         "query": "invasive ductal carcinoma",
#         "type": "phrase"
#       }
#     }
#   },
#   "aggs": {}
# }
    res = es.search(index='series', body={
            'query': {
                'match': {
                    'title': {
                        'query': phrase,
                        'operator': 'and'
                    }
                }
            }
        })
    hits = res['hits']['hits']
    for hit in hits:
        yield hit['_score'], hit['_source']['accession']
    

In [36]:
list(search_phrase('invasive ductal carcinoma'))

[(5.2213545, 'GSE2048'),
 (3.7318954, 'GSE33692'),
 (3.729539, 'GSE21422'),
 (3.729539, 'GSE51233'),
 (3.690568, 'GSE21444'),
 (2.9836311, 'GSE26304'),
 (2.8916152, 'GSE31892'),
 (2.8916152, 'GSE41228'),
 (2.6123269, 'GSE41227'),
 (2.6106772, 'GSE41198')]

In [80]:
diseases = list(meta_set['disease'].unique())
len(diseases)

161

In [82]:
from collections import defaultdict

annotations = defaultdict(list)
for disease in diseases:
    for score, accession in search_phrase(disease):
        annotations[accession].append((score, disease))
        
len(annotations)

812

In [86]:
meta_set.accession.unique().shape

(303,)

In [90]:
series_diseases = meta_set.groupby('accession')['disease'].apply(set)
series_diseases[:10]

accession
GSE10245    set([lung adenocarcinoma, lung squamous cell c...
GSE10334                                 set([periodontitis])
GSE10445                                   set([lung cancer])
GSE10714                            set([colorectal adenoma])
GSE10810                                 set([breast cancer])
GSE10927    set([adrenocortical carcinoma, adrenal cortica...
GSE10961                             set([colorectal cancer])
GSE10971               set([fallopian tube serous carcinoma])
GSE11001                                 set([breast cancer])
GSE11151    set([kidney clear cell sarcoma, renal cell car...
Name: disease, dtype: object

In [91]:
annotations_v = []
annotations_i = []
for accession, dis in annotations.items():
    annotations_v.append( set(dis))
    annotations_i.append(accession)

annotations_s = pd.DataFrame(pd.Series(data=annotations_v, index=annotations_i, name='diseases'))
annotations_s[:10]

,diseases
GSE11130,"set([(4.4734416, scleroderma)])"
GSE41033,"set([(2.7783904, brain glioma)])"
GSE56897,"set([(2.2606993, colon adenoma)])"
GSE17820,"set([(2.9952033, estrogen-receptor positive br..."
GSE29283,"set([(3.625551, hepatocellular carcinoma)])"
GSE20141,"set([(3.1077766, Parkinson's disease)])"
GSE40328,"set([(3.4131005, amyotrophic lateral sclerosis)])"
GSE2138,"set([(3.5908136, colon cancer)])"
GSE50115,"set([(2.7723703, colorectal adenocarcinoma), (..."
GSE19332,"set([(3.4131005, amyotrophic lateral sclerosis)])"


In [92]:
scores = pd.DataFrame.from_records([series_diseases]).T
scores.columns = ['diseases_correct']
scores = pd.merge(scores, annotations_s, left_index=True, right_index=True)
scores[:10]

,diseases_correct,diseases
GSE20141,set([Parkinson's disease]),"set([(3.1077766, Parkinson's disease)])"
GSE19332,set([amyotrophic lateral sclerosis]),"set([(3.4131005, amyotrophic lateral sclerosis)])"
GSE48574,set([myopathy]),"set([(2.2367208, myopathy)])"
GSE5081,set([gastritis]),"set([(2.8858016, gastritis)])"
GSE19314,set([sarcoidosis]),"set([(2.7259197, sarcoidosis)])"
GSE13732,set([multiple sclerosis]),"set([(3.572657, multiple sclerosis)])"
GSE50790,set([psoriasis]),"set([(2.5784898, psoriasis)])"
GSE37265,set([aphthous stomatitis]),"set([(3.4079835, aphthous stomatitis)])"
GSE9844,set([tongue squamous cell carcinoma]),"set([(4.571173, tongue squamous cell carcinoma)])"
GSE45512,set([alopecia areata]),"set([(5.963971, alopecia areata)])"


In [93]:
def match(diseases_correct, diseases):
    m = 0
    for score, d in diseases:
        if d in diseases_correct:
            m += 1
            
    return m/len(diseases_correct), (len(diseases) - m)/len(diseases)
scores['match'] = scores.apply(lambda row: match(row['diseases_correct'], row['diseases'])[0], axis=1)
scores['garb'] = scores.apply(lambda row: match(row['diseases_correct'], row['diseases'])[1], axis=1)
scores['str.diseases_correct'] = scores['diseases_correct'].map(', '.join)
scores['str.diseases'] = scores['diseases'].map(lambda x: ', '.join(a[1] for a in x))
scores[:10]

,diseases_correct,diseases,match,garb,str.diseases_correct,str.diseases
GSE20141,set([Parkinson's disease]),"set([(3.1077766, Parkinson's disease)])",1,0,Parkinson's disease,Parkinson's disease
GSE19332,set([amyotrophic lateral sclerosis]),"set([(3.4131005, amyotrophic lateral sclerosis)])",1,0,amyotrophic lateral sclerosis,amyotrophic lateral sclerosis
GSE48574,set([myopathy]),"set([(2.2367208, myopathy)])",1,0,myopathy,myopathy
GSE5081,set([gastritis]),"set([(2.8858016, gastritis)])",1,0,gastritis,gastritis
GSE19314,set([sarcoidosis]),"set([(2.7259197, sarcoidosis)])",1,0,sarcoidosis,sarcoidosis
GSE13732,set([multiple sclerosis]),"set([(3.572657, multiple sclerosis)])",1,0,multiple sclerosis,multiple sclerosis
GSE50790,set([psoriasis]),"set([(2.5784898, psoriasis)])",1,0,psoriasis,psoriasis
GSE37265,set([aphthous stomatitis]),"set([(3.4079835, aphthous stomatitis)])",1,0,aphthous stomatitis,aphthous stomatitis
GSE9844,set([tongue squamous cell carcinoma]),"set([(4.571173, tongue squamous cell carcinoma)])",1,0,tongue squamous cell carcinoma,tongue squamous cell carcinoma
GSE45512,set([alopecia areata]),"set([(5.963971, alopecia areata)])",1,0,alopecia areata,alopecia areata


In [94]:
scores[(scores['match'] < 1.0)][['match','garb','str.diseases_correct','str.diseases']]

,match,garb,str.diseases_correct,str.diseases
GSE31546,0.000000,1.000000,lung adenocarcinoma,lung cancer
GSE4183,0.000000,1.000000,"colorectal adenoma, colorectal cancer, inflamm...",colon adenoma
GSE15960,0.500000,0.000000,"colorectal adenoma, colorectal cancer",colorectal adenoma
GSE26049,0.666667,0.333333,"myelofibrosis, polycythemia Vera, hemorrhagic ...","polycythemia, polycythemia Vera, myelofibrosis"
GSE22242,0.000000,1.000000,colorectal cancer,colorectal adenoma
GSE11499,0.500000,0.000000,"Crohn's disease, ulcerative colitis",Crohn's disease
GSE21422,0.500000,0.500000,"invasive ductal carcinoma, ductal carcinoma in...","breast ductal carcinoma, invasive ductal carci..."
GSE36807,0.500000,0.000000,"Crohn's disease, ulcerative colitis",Crohn's disease
GSE30806,0.000000,1.000000,inclusion body myopathy with Paget disease of ...,myopathy
GSE22356,0.500000,0.000000,"pulmonary hypertension, scleroderma",scleroderma


In [95]:
scores[(scores['garb'] > 0.0)][['match','garb','str.diseases_correct','str.diseases']]

,match,garb,str.diseases_correct,str.diseases
GSE31546,0.000000,1.000000,lung adenocarcinoma,lung cancer
GSE4183,0.000000,1.000000,"colorectal adenoma, colorectal cancer, inflamm...",colon adenoma
GSE16538,1.000000,0.500000,pulmonary sarcoidosis,"sarcoidosis, pulmonary sarcoidosis"
GSE26049,0.666667,0.333333,"myelofibrosis, polycythemia Vera, hemorrhagic ...","polycythemia, polycythemia Vera, myelofibrosis"
GSE22242,0.000000,1.000000,colorectal cancer,colorectal adenoma
GSE21422,0.500000,0.500000,"invasive ductal carcinoma, ductal carcinoma in...","breast ductal carcinoma, invasive ductal carci..."
GSE8514,1.000000,0.500000,adrenal adenoma,"adrenal adenoma, adenoma"
GSE54043,1.000000,0.500000,eosinophilic gastritis,"gastritis, eosinophilic gastritis"
GSE44971,1.000000,0.500000,pilocytic astrocytoma,"astrocytoma, pilocytic astrocytoma"
GSE30806,0.000000,1.000000,inclusion body myopathy with Paget disease of ...,myopathy


In [96]:
print(scores[(scores['garb'] > 0.0) |(scores['match'] < 1.0)].shape[0], scores.shape[0])

26 102


In [76]:
scores.loc['GSE32924']['str.diseases']

'atopic dermatitis, atopic dermatitis '